In [1]:
# # IMPORTS
# astronomy
from astropy.io import fits
# plotting 
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.pyplot as plt
import matplotlib as mpl
# data 
import numpy as np
# file IO
import csv
# my python files 
import match
import v2_AGN_DataAndPlotting as adp

# Read Data

In [2]:
# read AGN IDs
agnIDs_file = adp.ReadFile(adp.path_csv+'\\v2_AGNid_Xray_MIR.csv')
# convert agnIDs into a dictionary of numpy arrays 
agnIDs = { 
    'MIR'  : np.array(agnIDs_file[0], dtype=int), 
    'XRAY' : np.array(agnIDs_file[1], dtype=int) 
    }
# print info
print('Number of MIR AGN:\t',    len(agnIDs['MIR']))
print('Number of Xray AGN:\t',   len(agnIDs['XRAY']))

Cropped:  ['Row 1 = MIR selected AGN (Donley et al, 2012) with id, redshift, IRAC, and 24um (n=900). Row 2 = X-ray selected AGN (Lx > 10^43 erg/s) with id, redshift, IRAC, and 24um (n=1775). ']
Number of MIR AGN:	 900
Number of Xray AGN:	 1775


In [3]:
# open COSMOS 2020 catalog file and get data 
with fits.open(adp.path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits') as hdul20:
    data20 = hdul20[1].data
    cols20 = hdul20[1].columns

In [4]:
# get IRAC data (unit = 'uJy')
ch1_data20  = np.array(data20['SPLASH_CH1_FLUX']) # 3.5686 um
print('.', end='')
ch2_data20  = np.array(data20['SPLASH_CH2_FLUX']) # 4.5067 um
print('.', end='')
ch3_data20  = np.array(data20['SPLASH_CH3_FLUX']) # 5.7788 um
print('.', end='')
ch4_data20  = np.array(data20['SPLASH_CH4_FLUX']) # 7.9958 um 
print('.', end='')
# get ID
id_data20 = np.array(data20['ID_COSMOS2015'])
print('.', end='')

.....

In [5]:
# open COSMOS 2016 catalog file and get data 
with fits.open(adp.path_cat+'\chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul16:
    data16 = hdul16[1].data
    cols16 = hdul16[1].columns

In [6]:
# get data arrays from columns
id_data16 = np.array(data16['id_k_uv'])
print('.', end='')
L0510_data16 = np.array(data16['Lx_0510']) # x-ray luminosity in the 0.5-10 KeV band
print('.', end='')
k0510_data16 = np.array(data16['abs_corr_0510'])  # absorbtion correction coefficient in the 0.5-10 KeV band
print('.', end='')

...

# Tricolor color vs color 

In [9]:
def TwoToTri(irID,xrID):
    # match agn ids
    key_ir, key_xr = match.match(irID, xrID)
    # initialize
    mask_ir = np.ones(irID.size, dtype=bool)
    mask_xr = np.ones(xrID.size, dtype=bool)
    # set matched index to false
    mask_ir[key_ir] = False
    mask_xr[key_xr] = False
    # remove xray selected agn from ir selected list
    irID_exc = irID[mask_ir]
    # remove ir selected agn from xray selected list 
    xrID_exc = xrID[mask_xr]
    # create list of ir/xray match
    ixrID_inc = irID[key_ir]
    # create and return dict of list of agn groups
    idDict = {
        'RED' : irID_exc,
        'YEL' : ixrID_inc,
        'BLU' : xrID_exc
    }
    return(idDict)

In [10]:
triID = TwoToTri(agnIDs['MIR'],agnIDs['XRAY'])

# print info 
print('Number of IR only:\t',   len(triID['RED']))
print('Number of Xray only:\t', len(triID['BLU']))
print('Number of matches:\t',   len(triID['YEL']))

Number of IR only:	 352
Number of Xray only:	 1227
Number of matches:	 548


In [11]:
# match IDs to data20 
key_data20_red, key_red = match.match(id_data20,triID['RED'])
key_data20_yel, key_yel = match.match(id_data20,triID['YEL'])
key_data20_blu, key_blu = match.match(id_data20,triID['BLU'])
# calculate colors 
xy_IRAC_tri = []
for i in range(len(triID)) : 
    x = np.log10(   ch3_data20[i]    /   ch1_data20[i]    )     # x = log10 ( f_5.6um / f_3.6um )
    y = np.log10(   ch4_data20[i]    /   ch2_data20[i]    )     # y = log10 ( f_8.0um / f_4.5 um )
    xy_IRAC_tri.append((x,y))

In [ ]:
# use dicts! no more REDYELBLU constants